# MiniZinc and IPython

The $n$-queens problem (see `models/queens.mzn`)

In [1]:
with open('models/queens.mzn','r') as file:
    queens_model = file.read()
    print(queens_model)

int: n;
array [1..n] of var 1..n: q;
include "alldifferent.mzn";
constraint alldifferent(q);
constraint alldifferent(i in 1..n)(q[i] + i);
constraint alldifferent(i in 1..n)(q[i] - i);
solve satisfy;



## Command line

In [2]:
for i in range(1,10+1):
    !minizinc models/queens.mzn -D n={i};

q = array1d(1..1, [1]);
----------
=====UNSATISFIABLE=====
=====UNSATISFIABLE=====
q = array1d(1..4, [3, 1, 4, 2]);
----------
q = array1d(1..5, [4, 2, 5, 3, 1]);
----------
q = array1d(1..6, [5, 3, 1, 6, 4, 2]);
----------
q = array1d(1..7, [6, 4, 2, 7, 5, 3, 1]);
----------
q = array1d(1..8, [4, 2, 7, 3, 6, 8, 5, 1]);
----------
q = array1d(1..9, [5, 7, 9, 4, 2, 8, 6, 3, 1]);
----------
q = array1d(1..10, [8, 5, 2, 4, 10, 7, 9, 6, 3, 1]);
----------


## Python minizinc package

In [3]:
import minizinc

In [4]:
model = minizinc.Model()
model.add_string(queens_model)

In [5]:
gecode = minizinc.Solver.lookup("gecode")
inst = minizinc.Instance(gecode, model)

In [6]:
# # RuntimeError: asyncio.run() cannot be called from a running event loop
# for i in range(1,10+1):
#     inst['n'] = i
#     result = inst.solve()
#     print(result)

## IPython MiniZinc extension

[IPython MiniZinc extension](https://github.com/MiniZinc/iminizinc)

In [7]:
%load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.5.0, build 198486215
Copyright (C) 2014-2020 Monash University, NICTA, Data61


In [8]:
%%minizinc??

In [10]:
n=8

In [11]:
%%minizinc
int: n;
array [1..n] of var 1..n: q;
include "alldifferent.mzn";
constraint alldifferent(q);
constraint alldifferent(i in 1..n)(q[i] + i); 
constraint alldifferent(i in 1..n)(q[i] - i);
solve satisfy;  

{'q': [4, 2, 7, 3, 6, 8, 5, 1]}

### Bind MiniZinc variables to Python variables

In [14]:
q

[4, 2, 7, 3, 6, 8, 5, 1]

In [13]:
%%minizinc -m bind
int: n;
array [1..n] of var 1..n: q;
include "alldifferent.mzn";
constraint alldifferent(q);
constraint alldifferent(i in 1..n)(q[i] + i); 
constraint alldifferent(i in 1..n)(q[i] - i);
solve satisfy;  

In [15]:
type(q)

list

In [16]:
print(q)

[4, 2, 7, 3, 6, 8, 5, 1]


### Store and reuse model

In [17]:
%%mzn_model queens
int: n;
array [1..n] of var 1..n: q;
include "alldifferent.mzn";
constraint alldifferent(q);
constraint alldifferent(i in 1..n)(q[i] + i); 
constraint alldifferent(i in 1..n)(q[i] - i);
solve satisfy;

In [18]:
for n in range(1,10+1):
    print(f"n={n}:", end="")
    result = %minizinc queens
    if result is not None:
        print(result['q'])    

n=1:[1]
n=2:Solver output:
The problem is infeasible
n=3:Solver output:
The problem is infeasible
n=4:[3, 1, 4, 2]
n=5:[4, 2, 5, 3, 1]
n=6:[5, 3, 1, 6, 4, 2]
n=7:[6, 4, 2, 7, 5, 3, 1]
n=8:[4, 2, 7, 3, 6, 8, 5, 1]
n=9:[5, 7, 9, 4, 2, 8, 6, 3, 1]
n=10:[8, 5, 2, 4, 10, 7, 9, 6, 3, 1]


### Count the number of solutions

In [19]:
counts = {}
for n in range(4,12+1):    
    print(f"n={n}", end="\r")
    result = %minizinc queens -a
    if result is None:
        counts[n] = 0
    else:        
        counts[n] = len(result)
        print(f"n={n}: {counts[n]} solutions")

n=4: 2 solutions
n=5: 10 solutions
n=6: 4 solutions
n=7: 40 solutions
n=8: 92 solutions
n=9: 352 solutions
n=10: 724 solutions
n=11: 2680 solutions
n=12: 14200 solutions


In [20]:
print(counts)

{4: 2, 5: 10, 6: 4, 7: 40, 8: 92, 9: 352, 10: 724, 11: 2680, 12: 14200}
